In [1]:
model_name = 'diffusion_model'

In [2]:
!pip install scipy==1.11.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [3]:
from keras.layers import Input, Dense, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import backend as K
from keras.callbacks import Callback
import tensorflow as tf

from scipy.stats import entropy
from scipy.linalg import sqrtm
from scipy.stats import pearsonr

from keras.applications.inception_v3 import InceptionV3

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import load_model

import cv2

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import entropy
from scipy.spatial.distance import cosine

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from scipy.linalg import sqrtm

In [4]:
import random

seed = 42

# Python's built-in random library
random.seed(seed)

# Scikit-learn (only affects some parts of scikit-learn)
from sklearn.utils import check_random_state
check_random_state(seed)

np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)

In [5]:
img_rows = 168
img_cols = 168
channels = 1

# Input image dimensions
img_shape = (img_rows, img_cols, channels)

In [6]:
# Set path for data source
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Load Diffusion/dataset/")

Mounted at /content/gdrive


In [7]:
# Set the path to the folder containing saved files
load_path = "./preprocessed_data_168x168_2years"

# Load saved NumPy arrays
X_train = np.load(os.path.join(load_path, 'X_train.npy'))
X_test = np.load(os.path.join(load_path, 'X_test.npy'))
y_train = np.load(os.path.join(load_path, 'y_train.npy'))
y_test = np.load(os.path.join(load_path, 'y_test.npy'))

# Load saved Pandas DataFrames
train_data = pd.read_pickle(os.path.join(load_path, 'train_data.pkl'))
test_data = pd.read_pickle(os.path.join(load_path, 'test_data.pkl'))
metadata = pd.read_pickle(os.path.join(load_path, 'metadata.pkl'))
metadata_original = pd.read_pickle(os.path.join(load_path, 'metadata_original.pkl'))

In [8]:
def merge_dummified_columns(df, dummified_columns, new_column_name):
    """
    Merges dummified columns into a single column in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the dummified columns.
        dummified_columns (list): A list of the names of the dummified columns.
        new_column_name (str): The name of the new column to create.

    Returns:
        pd.DataFrame: The DataFrame with the dummified columns merged into a single new column.
    """
    # Create a new column containing the column name where the value is 1 for each row
    df[new_column_name] = df[dummified_columns].idxmax(axis=1)

    # Drop the dummified columns
    df.drop(dummified_columns, axis=1, inplace=True)

    return df

test_metadata = metadata_original.loc[test_data.columns]

test_metadata = merge_dummified_columns(test_metadata,
                     dummified_columns=['chilledwater', 'electricity', 'gas', 'hotwater', 'steam'],
                     new_column_name='meter_type')
test_metadata = merge_dummified_columns(test_metadata,
                     dummified_columns=['Education', 'Entertainment/public assembly', 'Lodging/residential', 'Office', 'Public services'],
                     new_column_name='building_type')

test_metadata


,lat,lng,year,meter_type,building_type
electricity_Lamb_office_Caitlin_2016,51.497838,-3.186246,2016,electricity,Office
electricity_Rat_public_Kelle_2016,38.903504,-77.005349,2016,electricity,Public services
electricity_Rat_assembly_Ezequiel_2016,38.903504,-77.005349,2016,electricity,Entertainment/public assembly
electricity_Bear_assembly_Roxy_2016,37.871903,-122.260729,2016,electricity,Entertainment/public assembly
gas_Shrew_office_Rose_2017,51.499840,-0.124663,2017,gas,Office
...,...,...,...,...,...
steam_Moose_education_Omar_2016,45.421500,-75.697200,2016,steam,Education
electricity_Rat_public_Chrissy_2017,38.903504,-77.005349,2017,electricity,Public services
electricity_Hog_office_Merilyn_2016,44.978782,-93.255398,2016,electricity,Office
steam_Bull_assembly_Amalia_2016,30.267200,-97.743100,2016,steam,Entertainment/public assembly


In [9]:
def calculate_rmse(true_data, predicted_data):
    return np.sqrt(mean_squared_error(true_data, predicted_data))

def calculate_mae(true_data, predicted_data):
    return mean_absolute_error(true_data, predicted_data)

def calculate_r2(true_data, predicted_data):
    return r2_score(true_data, predicted_data)

def calculate_kl_divergence(true_data, predicted_data):
    return entropy(true_data+0.00000001, predicted_data+0.00000001)

def calculate_cosine_similarity(true_data, predicted_data):
    return 1 - cosine(true_data, predicted_data)

In [10]:
def evaluate_model(test_data, generated_series):
    metrics_result = {
        'rmse': [],
        'mae': [],
        'r2': [],
        'kl_divergence': [],
        'cosine_similarity': [],
        'pearson_correlation': []
    }

    for i in range(test_data.shape[1]):
        try:
            true_data = test_data[:, i]
            predicted_data = generated_series[:, i]
        except:
            true_data = test_data.values[:, i]
            predicted_data = generated_series.values[:, i]

        # Calculate Pearson correlation
        pearson_corr, _ = pearsonr(true_data, predicted_data)

        metrics_result['rmse'].append(calculate_rmse(true_data, predicted_data))
        metrics_result['mae'].append(calculate_mae(true_data, predicted_data))
        metrics_result['r2'].append(calculate_r2(true_data, predicted_data))
        metrics_result['kl_divergence'].append(calculate_kl_divergence(true_data, predicted_data))
        metrics_result['cosine_similarity'].append(calculate_cosine_similarity(true_data, predicted_data))
        metrics_result['pearson_correlation'].append(pearson_corr)

    return metrics_result

# Function to calculate FID between two sets of images
def calculate_fid(model, images1, images2):
    # Calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)

    # Calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # Calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # Calculate sqrt of product between covariances
    covmean = sqrtm(sigma1.dot(sigma2))

    # Check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Calculate the FID score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [11]:
# Specify the path where the datasets are saved
generation_path = "./generation/" + model_name + '_168x168'

# Initialize an empty DataFrame to store metrics
df_metrics = []

for file_name in os.listdir(generation_path):
  # Load the dataset
  generated_images = np.load(os.path.join(generation_path, file_name))
  images_seed = int(file_name.split('.')[0].split('_')[-1])

  # Set the target shape
  target_shape = (generated_images.shape[0], 52, 168, 1)

  # Initialize an array to hold the resized images
  resized_images = np.zeros(target_shape)

  # Loop through each image to resize it
  for i in range(generated_images.shape[0]):
      resized_image = cv2.resize(generated_images[i], (168, 52))  # Note the dimensions are (width, height)
      resized_images[i] = np.expand_dims(resized_image, axis=-1)  # Add back the last dimension

  # Assume resized_images has a shape of (905, 52, 168, 1)
  flattened_images = resized_images.reshape(generated_images.shape[0], -1).T  # Transpose to get shape (8736, generated_images.shape[0])

  generated_series = pd.DataFrame(flattened_images)
  generated_series.columns = test_data.columns

  generated_series = (generated_series-generated_series.min())/(generated_series.max()-generated_series.min())
  test_data = (test_data-test_data.min())/(test_data.max()-test_data.min())

  test_data_resampled = test_data.rolling(24).mean()[23::24]
  generated_series_resampled = generated_series.rolling(24).mean()[23::24]

  evaluation_results = evaluate_model(test_data, generated_series)
  evaluation_results_resampled = evaluate_model(test_data_resampled, generated_series_resampled)

  df_evaluation_results = pd.DataFrame(evaluation_results)
  df_evaluation_results_resampled = pd.DataFrame(evaluation_results_resampled)
  df_evaluation_results_resampled.columns = df_evaluation_results_resampled.columns+'_daily_avg'

  # Prepare the Inception v3 model
  model = InceptionV3(include_top=False, pooling='avg', weights="imagenet", input_shape=(99,99,3))

  # Convert to RGB and upscale to the necessary input size for the Inception model
  X_test_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_test))
  generated_images_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(generated_images))

  X_test_upscaled = tf.image.resize(X_test_rgb, [99, 99])
  generated_images_upscaled = tf.image.resize(generated_images_rgb, [99, 99])

  # Calculate FID
  fid_score = calculate_fid(model, X_test_upscaled, generated_images_upscaled)
  print('FID: '+str(fid_score))

  metrics = pd.concat([df_evaluation_results.mean(), df_evaluation_results_resampled.mean()])
  metrics['fid_score'] = fid_score
  metrics['images_seed'] = images_seed
  metrics['model'] = model_name
  metrics = metrics.to_frame().T

  # Append the metrics to df_metrics
  df_metrics.append(metrics)

29/29 [==============================] - 16s 555ms/step
FID: 517.7867001449742
29/29 [==============================] - 17s 576ms/step
FID: 512.1445150587934
29/29 [==============================] - 17s 573ms/step
FID: 516.6871320211224
29/29 [==============================] - 16s 542ms/step
FID: 513.6530691368752
29/29 [==============================] - 15s 534ms/step
FID: 512.779852066052
29/29 [==============================] - 15s 517ms/step
FID: 514.199088209566
29/29 [==============================] - 16s 548ms/step
FID: 521.3273984629789
29/29 [==============================] - 16s 521ms/step
FID: 512.2663605654045
29/29 [==============================] - 16s 545ms/step
FID: 515.3954691276849
29/29 [==============================] - 16s 546ms/step
FID: 528.0308168698559
29/29 [==============================] - 16s 554ms/step
FID: 522.3458433276011
29/29 [==============================] - 16s 546ms/step
FID: 524.4910823117362
29/29 [==============================] - 17s 597ms/ste

In [12]:
df_metrics = pd.concat(df_metrics, ignore_index=True)
df_metrics

,rmse,mae,r2,kl_divergence,cosine_similarity,pearson_correlation,rmse_daily_avg,mae_daily_avg,r2_daily_avg,kl_divergence_daily_avg,cosine_similarity_daily_avg,pearson_correlation_daily_avg,fid_score,images_seed,model
0,0.244876,0.200829,-3.438515,0.394846,0.806622,0.429623,0.200504,0.169569,-54.486312,0.24564,0.869799,0.458994,517.7867,1.0,diffusion_model
1,0.245811,0.201123,-3.96638,0.400144,0.805375,0.42258,0.201191,0.169687,-72.161198,0.249009,0.868746,0.452031,512.144515,2.0,diffusion_model
2,0.246997,0.202186,-3.231512,0.404612,0.803461,0.418322,0.203059,0.171561,-55.462353,0.252931,0.866385,0.440564,516.687132,3.0,diffusion_model
3,0.246654,0.201918,-3.380347,0.404015,0.803037,0.421451,0.202299,0.170656,-58.520661,0.254024,0.865242,0.440642,513.653069,4.0,diffusion_model
4,0.244766,0.200014,-3.196604,0.403005,0.804793,0.425432,0.200813,0.16941,-55.773813,0.254909,0.866135,0.450104,512.779852,5.0,diffusion_model
5,0.244916,0.200764,-2.982125,0.399948,0.805831,0.433424,0.200371,0.169467,-47.093638,0.249365,0.868678,0.459755,514.199088,6.0,diffusion_model
6,0.247307,0.202898,-3.142238,0.409961,0.801153,0.416386,0.202877,0.17166,-52.374453,0.257968,0.863645,0.437406,521.327398,7.0,diffusion_model
7,0.246576,0.20224,-3.568715,0.404886,0.802713,0.411928,0.202871,0.171678,-59.444721,0.25584,0.863837,0.427003,512.266361,8.0,diffusion_model
8,0.244471,0.199887,-3.110968,0.397083,0.805793,0.425788,0.20004,0.168665,-51.50081,0.246662,0.868643,0.44877,515.395469,9.0,diffusion_model
9,0.244184,0.19936,-3.315773,0.401162,0.805245,0.427535,0.200422,0.168582,-54.96043,0.252259,0.866971,0.452416,528.030817,10.0,diffusion_model


In [13]:
generation_path = "./evaluations"
df_metrics.to_csv(os.path.join(generation_path, model_name+'_metrics.csv'),index=False)

In [14]:
os.path.join(generation_path, model_name+'_metrics.csv')

'./evaluations/diffusion_model_metrics.csv'